# Interact+InspectDR: Bode Plot Example

In [ ]:
using InspectDR

#When MIME"image/svg+xml" is disabled, Jupyter eventually requests PNG inline graphics:
#(SVG outputs do not render well in notebooks for some reason...)
InspectDR.defaults.rendersvg = false

include("BodePlots.jl") #Defines BodePlots.* utilities.

nothing

# Setup: Interact, Reactive, and Signals
(Also create initial plot object)

In [ ]:
using Interact, Reactive

fopt = [("kHz", 1.0e3), ("MHz", 1.0e6), ("GHz", 1.0e9)] #Make sure to use Float64
freqmap = Dict{Float64, String}(v=>s for (s,v) in fopt)
fmult = Signal(Float64, 1e9) #Frequency multiplier
annot = Signal(Bool, true) #Annotate plot
fmax = Signal(Float64, 10) #Maximum plot frequency
G = Signal(Float64, 20) #Gain (dB)
fz = Signal(Float64, value(fmax)) #Zero frequency
fp1 = Signal(Float64, .05) #Pole 1 frequency
fp2 = Signal(Float64, .9) #Pole 2 frequency
pobj = BodePlots.new() #Create initial plot object

getfreqrange(fmax) = 1e-3:1e-3:fmax
function update_range(widget, fmax)
    widget.range = getfreqrange(fmax)
    update_view(widget)
end
function frequency_slider(sig::Signal, fmax::Signal; label::String="")
    sld = slider(getfreqrange(value(fmax)), value=value(sig), label=label, signal=sig)
    display(sld) #Must desplay before building updater, for some reason.
    updater = map(_fmax->update_range(sld, _fmax), fmax)
    return (sld, updater)
end

nothing

# Interact/Reactive Control: Inline Plots
Use sliders/text boxes to change pole frequency

## TODO

 - Find good way to support log-spaced ranges for frequencies.
 - Find out why txt_fmax will not display when specifying `signal=`.
 - Find out why chk_annot hangs on 2nd execution of code block when specifying `signal=`.
 - Find out why one cannot display all widgets in a single statement (must display in stages for some reason).
 - Improve widgets layout (`hstack`?) once supported by Interact.

In [ ]:
tgl_fmult = togglebuttons(fopt, value_label=freqmap[value(fmult)], signal=fmult)
#chk_annot = checkbox(value(annot), label="Display annotation", signal=annot) #Hangs on 2nd execution of block
chk_annot = checkbox(value(annot), label="Display annotation")
annot = signal(chk_annot) #Hack to avoid hangups
map(display, [tgl_fmult, chk_annot])

#txt_fmax = textbox(value(fmax), label="max freq", signal=fmax) #Will not display - notebook hangs??
txt_fmax = textbox(value(fmax), label="max freq") #This one does not cause hangups
fmax = signal(txt_fmax) #Hack to get fmax/txt_fmax working
display(txt_fmax) #Will hang session using 1st method, for some reason

#Gain slider:
sld_G = slider(-10:1.0:100, value=value(G), label="Gain (dB)", signal=G)
display(sld_G)

#Pole/zero sliders:
(sld_fz, updater_fz) = frequency_slider(fz, fmax, label="zero")
(sld_fp1, updater_fp1) = frequency_slider(fp1, fmax, label="pole 1")
(sld_fp2, updater_fp2) = frequency_slider(fp2, fmax, label="pole 2")
#map(display, [sld_fz, sld_fp1, sld_fp2]) #Display now in frequency_slider()

#Build widget list to control plot for future code blocks
#TODO: Why do widgets have to be displayed in stages?
widgetlist = [tgl_fmult, chk_annot, txt_fmax, sld_G, sld_fz, sld_fp1, sld_fp2]

#Widget-controlled plot:
updater_plot = map(fmult, fmax, G, fz, fp1, fp2, annot) do s, _fmax, _G, _fz, _fp1, _fp2, a
    BodePlots.update(pobj, s*1e-3, _fmax*s, _G, _fz*s, _fp1*s, _fp2*s, a)
end
display(updater_plot)

nothing

# Interact/Reactive Control: Inspect/Gtk GUI
(Re-display GUI controls for convenience)

In [ ]:
gtkgui = display(InspectDR.GtkDisplay(), pobj) #Display plot in Gtk GUI.

#Refresh GUI when plot changes:
updater_gtkgui = map((p)->InspectDR.refresh(gtkgui), updater_plot)

#Re-display GUI controls, for convenience:
map(display, widgetlist)

nothing

# Demo complete!